## KNN

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import gc


In [2]:
# If importing is giving you an issue, don't import mixed data type columns 
#good_import_cols1 = pd.Series(range(0,57))
#good_import_cols = good_import_cols1.drop([22,32,34,49,55])


In [3]:
# If importing is giving you an issue, only import good columns  
train_df1 = pd.read_csv("train_2016_v2.csv", parse_dates=["transactiondate"])
prop = pd.read_csv("properties_2016.csv") #, usecols=good_import_cols)

C:\Users\board\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

In [5]:
missing_df = prop.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count']>0]

missing_df['missing_ratio'] = missing_df['missing_count'] / prop.shape[0]
missing_df.loc[missing_df['missing_ratio']>0.75]

,column_name,missing_count,missing_ratio
2,architecturalstyletypeid,2979156,0.997970
3,basementsqft,2983589,0.999455
6,buildingclasstypeid,2972588,0.995769
9,decktypeid,2968121,0.994273
10,finishedfloor1squarefeet,2782500,0.932093
13,finishedsquarefeet13,2977545,0.997430
14,finishedsquarefeet15,2794419,0.936086
15,finishedsquarefeet50,2782500,0.932093
16,finishedsquarefeet6,2963216,0.992630
18,fireplacecnt,2672580,0.895272


Impute mean values where there are missing values  
Will impute for where less than 75% are missing 

In [6]:
train_df = train_df1.merge(prop, on='parcelid', how = 'left')
mean_values = train_df.mean(axis=0)
train_df_new = train_df.fillna(mean_values, inplace=True)


In [7]:
#cols_to_drop = train_df.dtypes[train_df.dtypes != "float64"].index
train_df2 = train_df.drop(['propertycountylandusecode', 'propertyzoningdesc', 'taxdelinquencyflag', 'hashottuborspa','fireplaceflag' ], axis = 1)


In [8]:
missing_df2 = train_df2.isnull().sum(axis=0).reset_index()
missing_df2.columns = ['count', 'val']
missing_df2[missing_df2['val'] > 0]

,count,val


In [9]:
train_df2.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,...,pooltypeid10,pooltypeid2,pooltypeid7,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.000000,7.229885,713.581395,2.0,3.0,4.0,4.000000,2.0,66.0,1347.974037,1684.0,1684.0,1404.545455,2380.090067,1355.546674,2302.546318,6037.0,1.187884,2.0,1.812005,345.521228,2.000000,...,1.0,1.0,1.0,261.0,6.037107e+07,12447.0,3101.0,31817.000000,96370.0,0.0,7.0,1.008743,6.010033,1.000000,310.142101,311.694737,1959.0,1.440739,122754.0,360170.0,2015.0,237416.0,6735.88,13.402692,6.037107e+13
1,14366692,-0.1684,2016-01-01,1.816372,7.229885,713.581395,3.5,4.0,4.0,5.565407,3.5,66.0,1347.974037,2263.0,2263.0,1404.545455,2380.090067,1355.546674,2302.546318,6059.0,1.187884,3.0,2.000000,468.000000,3.926979,...,1.0,1.0,1.0,261.0,6.059052e+07,32380.0,1286.0,190646.540237,96962.0,0.0,7.0,1.000000,6.010033,1.110414,310.142101,311.694737,2014.0,1.440739,346458.0,585529.0,2015.0,239071.0,10153.02,13.402692,6.049151e+13
2,12098116,-0.0040,2016-01-01,1.000000,7.229885,713.581395,3.0,2.0,4.0,4.000000,3.0,66.0,1347.974037,2217.0,2217.0,1404.545455,2380.090067,1355.546674,2302.546318,6037.0,1.187884,3.0,1.812005,345.521228,2.000000,...,1.0,1.0,1.0,261.0,6.037464e+07,47019.0,3101.0,275411.000000,96293.0,0.0,7.0,1.008743,6.010033,1.000000,310.142101,311.694737,1940.0,1.440739,61994.0,119906.0,2015.0,57912.0,11484.48,13.402692,6.037464e+13
3,12643413,0.0218,2016-01-02,1.000000,7.229885,713.581395,2.0,2.0,4.0,4.000000,2.0,66.0,1347.974037,839.0,839.0,1404.545455,2380.090067,1355.546674,2302.546318,6037.0,1.187884,2.0,1.812005,345.521228,2.000000,...,1.0,1.0,1.0,266.0,6.037296e+07,12447.0,3101.0,54300.000000,96222.0,0.0,7.0,1.008743,6.010033,1.000000,310.142101,311.694737,1987.0,1.440739,171518.0,244880.0,2015.0,73362.0,3048.74,13.402692,6.037296e+13
4,14432541,-0.0050,2016-01-02,1.816372,7.229885,713.581395,2.5,4.0,4.0,5.565407,2.5,66.0,1347.974037,2283.0,2283.0,1404.545455,2380.090067,1355.546674,2302.546318,6059.0,1.187884,2.0,2.000000,598.000000,3.926979,...,1.0,1.0,1.0,261.0,6.059042e+07,17686.0,1286.0,190646.540237,96961.0,8.0,7.0,1.000000,6.010033,1.110414,310.142101,311.694737,1981.0,2.000000,169574.0,434551.0,2015.0,264977.0,5488.96,13.402692,6.059042e+13


### One Hot Encoding -- Dummy Variables 

We know that some of the columns are categorical and should be treated as such  
We know that BuildingClassTypeID, TypeConstructionTypeID, ArchitecturalStyleTypeID, AirConditioningTypeID, HeatingOrSystemTypeID, PropertyLandUseTypeID, StoryTypeID are categorical variables 



In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
one_hot_colnames = ["buildingclasstypeid", "typeconstructiontypeid", "architecturalstyletypeid", "airconditioningtypeid", 
                    "heatingorsystemtypeid", "propertylandusetypeid", "storytypeid"]

In [12]:
one_hot_df = pd.DataFrame()
for col_name in one_hot_colnames:
    temp_one_hot_var1 = OneHotEncoder().fit_transform(train_df2[col_name].values.reshape(-1,1))
    temp_one_hot_var = temp_one_hot_var1.toarray()
    temp_one_hot_var_df1 = pd.DataFrame(data=temp_one_hot_var)
    temp_one_hot_var_df = temp_one_hot_var_df1.add_prefix(col_name+"_")
    one_hot_df = pd.concat([one_hot_df, temp_one_hot_var_df] , axis = 1)
print("Data Frame Size: " ,one_hot_df.shape)

Data Frame Size:  (90275, 44)


In [13]:
one_hot_df.head()

,buildingclasstypeid_0,typeconstructiontypeid_0,typeconstructiontypeid_1,typeconstructiontypeid_2,architecturalstyletypeid_0,architecturalstyletypeid_1,architecturalstyletypeid_2,architecturalstyletypeid_3,architecturalstyletypeid_4,architecturalstyletypeid_5,airconditioningtypeid_0,airconditioningtypeid_1,airconditioningtypeid_2,airconditioningtypeid_3,airconditioningtypeid_4,airconditioningtypeid_5,heatingorsystemtypeid_0,heatingorsystemtypeid_1,heatingorsystemtypeid_2,heatingorsystemtypeid_3,heatingorsystemtypeid_4,heatingorsystemtypeid_5,heatingorsystemtypeid_6,heatingorsystemtypeid_7,heatingorsystemtypeid_8,heatingorsystemtypeid_9,heatingorsystemtypeid_10,heatingorsystemtypeid_11,heatingorsystemtypeid_12,propertylandusetypeid_0,propertylandusetypeid_1,propertylandusetypeid_2,propertylandusetypeid_3,propertylandusetypeid_4,propertylandusetypeid_5,propertylandusetypeid_6,propertylandusetypeid_7,propertylandusetypeid_8,propertylandusetypeid_9,propertylandusetypeid_10,propertylandusetypeid_11,propertylandusetypeid_12,propertylandusetypeid_13,storytypeid_0
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


The dataset we will be using is train_df2. Let's take a last quick peak and check it for nulls 

In [14]:
train_df3 = train_df2.drop(one_hot_colnames, axis = 1)
train_df3 = pd.concat([train_df3, one_hot_df], axis =1)

In [15]:
train_df3.head()

,parcelid,logerror,transactiondate,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,latitude,longitude,lotsizesquarefeet,poolcnt,...,heatingorsystemtypeid_3,heatingorsystemtypeid_4,heatingorsystemtypeid_5,heatingorsystemtypeid_6,heatingorsystemtypeid_7,heatingorsystemtypeid_8,heatingorsystemtypeid_9,heatingorsystemtypeid_10,heatingorsystemtypeid_11,heatingorsystemtypeid_12,propertylandusetypeid_0,propertylandusetypeid_1,propertylandusetypeid_2,propertylandusetypeid_3,propertylandusetypeid_4,propertylandusetypeid_5,propertylandusetypeid_6,propertylandusetypeid_7,propertylandusetypeid_8,propertylandusetypeid_9,propertylandusetypeid_10,propertylandusetypeid_11,propertylandusetypeid_12,propertylandusetypeid_13,storytypeid_0
0,11016594,0.0276,2016-01-01,713.581395,2.0,3.0,4.000000,2.0,66.0,1347.974037,1684.0,1684.0,1404.545455,2380.090067,1355.546674,2302.546318,6037.0,1.187884,2.0,1.812005,345.521228,34280990.0,-118488536.0,7528.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,14366692,-0.1684,2016-01-01,713.581395,3.5,4.0,5.565407,3.5,66.0,1347.974037,2263.0,2263.0,1404.545455,2380.090067,1355.546674,2302.546318,6059.0,1.187884,3.0,2.000000,468.000000,33668120.0,-117677556.0,3643.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,12098116,-0.0040,2016-01-01,713.581395,3.0,2.0,4.000000,3.0,66.0,1347.974037,2217.0,2217.0,1404.545455,2380.090067,1355.546674,2302.546318,6037.0,1.187884,3.0,1.812005,345.521228,34136312.0,-118175032.0,11423.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,12643413,0.0218,2016-01-02,713.581395,2.0,2.0,4.000000,2.0,66.0,1347.974037,839.0,839.0,1404.545455,2380.090067,1355.546674,2302.546318,6037.0,1.187884,2.0,1.812005,345.521228,33755800.0,-118309000.0,70859.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,14432541,-0.0050,2016-01-02,713.581395,2.5,4.0,5.565407,2.5,66.0,1347.974037,2283.0,2283.0,1404.545455,2380.090067,1355.546674,2302.546318,6059.0,1.187884,2.0,2.000000,598.000000,33485643.0,-117700234.0,6000.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
print(train_df3.shape)
print(train_df3.columns[train_df3.isnull().sum(axis = 0)> 0])

(90275, 92)
Index([], dtype='object')


In [17]:
del train_df, train_df_new, train_df1, one_hot_df
gc.collect()

260

### Begin KNN Analysis 

Now that the data is in a nice format without nulls, we can begin the KNN analysis.  
We will start by scaling the data because KNN is sensitive to the scale of the data. Larger variables will seem more important even if they do not have the correct correlation with the output 

In [18]:
from sklearn.preprocessing import MinMaxScaler

In [19]:
train_df_scale = MinMaxScaler().fit_transform(train_df3.drop(['parcelid', 'transactiondate', 'logerror'],axis = 1))

In [20]:
scale_df_cols = train_df3.drop(['parcelid', 'transactiondate', 'logerror'],axis = 1).columns
train_df4 = pd.DataFrame(train_df_scale, columns = scale_df_cols) 
train_df4.describe()

,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,...,heatingorsystemtypeid_3,heatingorsystemtypeid_4,heatingorsystemtypeid_5,heatingorsystemtypeid_6,heatingorsystemtypeid_7,heatingorsystemtypeid_8,heatingorsystemtypeid_9,heatingorsystemtypeid_10,heatingorsystemtypeid_11,heatingorsystemtypeid_12,propertylandusetypeid_0,propertylandusetypeid_1,propertylandusetypeid_2,propertylandusetypeid_3,propertylandusetypeid_4,propertylandusetypeid_5,propertylandusetypeid_6,propertylandusetypeid_7,propertylandusetypeid_8,propertylandusetypeid_9,propertylandusetypeid_10,propertylandusetypeid_11,propertylandusetypeid_12,propertylandusetypeid_13,storytypeid_0
count,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.0,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.0,90275.000000,90275.0,90275.0,...,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.00000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.000000,90275.0
mean,0.421705,0.113974,0.189492,0.415037,0.068906,0.0,0.172006,0.077892,0.087125,0.660124,0.082056,0.157866,0.293605,0.160413,0.046971,0.065328,0.075500,0.047080,0.451080,0.659818,0.004152,0.0,0.285614,0.0,0.0,...,0.010745,0.171908,0.000022,0.000011,0.000011,0.000842,0.000022,0.000277,0.001074,0.011864,0.000188,0.000011,0.026320,0.006968,0.009737,0.000687,0.671692,0.00093,0.000122,0.003944,0.252728,0.000310,0.025854,0.000510,0.0
std,0.006485,0.050214,0.072277,0.137731,0.051040,0.0,0.023714,0.040668,0.044278,0.003926,0.009568,0.022346,0.013180,0.279236,0.039485,0.050359,0.014607,0.020952,0.179429,0.190499,0.016451,0.0,0.009324,0.0,0.0,...,0.103100,0.377303,0.004707,0.003328,0.003328,0.029003,0.004707,0.016639,0.032762,0.108273,0.013722,0.003328,0.160085,0.083181,0.098195,0.026198,0.469600,0.03049,0.011038,0.062674,0.434579,0.017609,0.158702,0.022568,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,0.421705,0.100000,0.125000,0.272727,0.052632,0.0,0.172006,0.052113,0.059367,0.660124,0.082056,0.157866,0.293605,0.000000,0.046971,0.052632,0.075500,0.047080,0.319793,0.547388,0.000831,0.0,0.285614,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,0.421705,0.100000,0.187500,0.415037,0.052632,0.0,0.172006,0.067945,0.077907,0.660124,0.082056,0.157866,0.293605,0.000000,0.046971,0.052632,0.075500,0.047080,0.461975,0.673256,0.001062,0.0,0.285614,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,0.421705,0.150000,0.250000,0.545455,0.105263,0.0,0.172006,0.091781,0.100495,0.660124,0.082056,0.157866,0.293605,0.297297,0.046971,0.105263,0.083333,0.047080,0.564393,0.806299,0.003150,0.0,0.285614,0.0,0.0,...,0.00

In [22]:
from sklearn.model_selection import train_test_split
Y = train_df3['logerror']
X_train, X_test, y_train, y_test = train_test_split(train_df4,Y, test_size = 0.25)

### KNN Regression 

Perform a cross validation with the KNN Regressor.  
It is commented out because it take a while to run. 

In [23]:
train_df4.head()

,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,...,heatingorsystemtypeid_3,heatingorsystemtypeid_4,heatingorsystemtypeid_5,heatingorsystemtypeid_6,heatingorsystemtypeid_7,heatingorsystemtypeid_8,heatingorsystemtypeid_9,heatingorsystemtypeid_10,heatingorsystemtypeid_11,heatingorsystemtypeid_12,propertylandusetypeid_0,propertylandusetypeid_1,propertylandusetypeid_2,propertylandusetypeid_3,propertylandusetypeid_4,propertylandusetypeid_5,propertylandusetypeid_6,propertylandusetypeid_7,propertylandusetypeid_8,propertylandusetypeid_9,propertylandusetypeid_10,propertylandusetypeid_11,propertylandusetypeid_12,propertylandusetypeid_13,storytypeid_0
0,0.421705,0.100,0.1875,0.272727,0.052632,0.0,0.172006,0.073970,0.084054,0.660124,0.082056,0.157866,0.293605,0.000000,0.046971,0.052632,0.075500,0.047080,0.637696,0.506793,0.001056,0.0,0.285614,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.421705,0.175,0.2500,0.415037,0.131579,0.0,0.172006,0.099433,0.112988,0.660124,0.082056,0.157866,0.293605,0.297297,0.046971,0.105263,0.083333,0.063769,0.222673,0.935216,0.000499,0.0,0.285614,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.421705,0.150,0.1250,0.272727,0.105263,0.0,0.172006,0.097410,0.110689,0.660124,0.082056,0.157866,0.293605,0.000000,0.046971,0.105263,0.075500,0.047080,0.539723,0.672410,0.001615,0.0,0.285614,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.421705,0.100,0.1250,0.272727,0.052632,0.0,0.172006,0.036809,0.041827,0.660124,0.082056,0.157866,0.293605,0.000000,0.046971,0.052632,0.075500,0.047080,0.282049,0.601638,0.010141,0.0,0.285614,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.421705,0.125,0.2500,0.415037,0.078947,0.0,0.172006,0.100312,0.113987,0.660124,0.082056,0.157866,0.293605,0.297297,0.046971,0.052632,0.083333,0.081482,0.099104,0.923236,0.000837,0.0,0.285614,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### K-Means Analysis 
We will use unsupervised learning to cluster the variables into groups. 


In [24]:
from sklearn.cluster import KMeans

In [27]:
kmeans1 = KMeans(n_clusters = 3).fit(X_train)

In [29]:
cluster_centers = pd.DataFrame(kmeans1.cluster_centers_, columns = train_df4.columns, index = [1,2,3])

In [30]:
cluster_centers.head()

,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,...,heatingorsystemtypeid_3,heatingorsystemtypeid_4,heatingorsystemtypeid_5,heatingorsystemtypeid_6,heatingorsystemtypeid_7,heatingorsystemtypeid_8,heatingorsystemtypeid_9,heatingorsystemtypeid_10,heatingorsystemtypeid_11,heatingorsystemtypeid_12,propertylandusetypeid_0,propertylandusetypeid_1,propertylandusetypeid_2,propertylandusetypeid_3,propertylandusetypeid_4,propertylandusetypeid_5,propertylandusetypeid_6,propertylandusetypeid_7,propertylandusetypeid_8,propertylandusetypeid_9,propertylandusetypeid_10,propertylandusetypeid_11,propertylandusetypeid_12,propertylandusetypeid_13,storytypeid_0
1,0.421705,0.074443,0.165931,0.528135,0.025834,0.0,0.172006,0.054128,0.061403,0.660124,0.082081,0.157866,0.293605,4.350567e-04,0.046955,0.025826,0.075479,0.047084,0.465379,0.645240,0.001812,0.0,0.285614,0.0,0.0,...,-2.014014e-15,9.987949e-01,-2.320870e-18,-2.320870e-18,-2.320870e-18,-1.019150e-16,-2.320870e-18,7.155734e-18,1.205130e-03,1.207368e-15,2.582422e-04,-2.320870e-18,0.002066,0.000258,-1.922074e-15,0.000258,0.943101,-1.967827e-16,-1.856696e-17,6.609296e-16,0.052681,0.000861,0.000516,-2.515349e-17,0.0
2,0.421718,0.117110,0.196749,0.426141,0.074491,0.0,0.171938,0.083244,0.091174,0.660152,0.082123,0.157761,0.293599,3.992267e-01,0.047041,0.065584,0.075575,0.047100,0.347693,0.708039,0.002358,0.0,0.285667,0.0,0.0,...,2.675524e-02,2.936103e-04,3.670129e-05,3.670129e-05,3.670129e-05,1.761662e-03,3.670129e-05,8.074283e-04,3.645088e-16,3.009506e-02,4.037142e-04,3.670129e-05,0.062686,0.016883,2.440636e-02,0.001174,0.613352,2.128675e-03,1.985445e-17,1.045987e-02,0.263185,0.000037,0.004000,1.247844e-03,0.0
3,0.421705,0.127086,0.192157,0.357841,0.081168,0.0,0.172006,0.082584,0.093836,0.660124,0.082057,0.157866,0.293605,-4.848899e-14,0.046971,0.081167,0.075500,0.047079,0.543650,0.619256,0.006794,0.0,0.285614,0.0,0.0,...,-6.869505e-16,4.204970e-14,3.201785e-18,3.201785e-18,3.201785e-18,2.586906e-16,3.201785e-18,-2.471439e-16,1.906941e-03,-6.021225e-15,-6.041717e-17,3.201785e-18,0.000139,0.000035,-2.482389e-15,0.000416,0.617537,3.467166e-05,2.773733e-04,7.467985e-16,0.324423,0.000347,0.056792,-2.721347e-17,0.0


In [31]:
cluster_centers.T[(cluster_centers.T[1] - cluster_centers.T[2]) > 0.02]

,1,2,3
buildingqualitytypeid,0.528135,0.426141,3.578407e-01
latitude,0.465379,0.347693,5.436500e-01
regionidcity,0.095406,0.070972,7.569345e-02
regionidcounty,0.998537,0.211648,1.000000e+00
airconditioningtypeid_0,0.998709,0.942269,9.999653e-01
heatingorsystemtypeid_4,0.998795,0.000294,4.204970e-14
propertylandusetypeid_6,0.943101,0.613352,6.175369e-01


In [32]:
cluster_centers.T[(cluster_centers.T[1] - cluster_centers.T[3]) > 0.02]

,1,2,3
buildingqualitytypeid,0.528135,0.426141,3.578407e-01
longitude,0.645240,0.708039,6.192556e-01
regionidneighborhood,0.245604,0.261401,2.231534e-01
heatingorsystemtypeid_4,0.998795,0.000294,4.204970e-14
propertylandusetypeid_6,0.943101,0.613352,6.175369e-01


In [33]:
# Cluster centers where the difference between the 2nd and 3rd is "high" 
cluster_centers.T[(cluster_centers.T[2] - cluster_centers.T[3]) > 0.02]

,1,2,3
buildingqualitytypeid,5.281350e-01,0.426141,3.578407e-01
fips,4.350567e-04,0.399227,-4.848899e-14
longitude,6.452403e-01,0.708039,6.192556e-01
rawcensustractandblock,4.670784e-03,0.399598,5.016786e-03
regionidneighborhood,2.456044e-01,0.261401,2.231534e-01
roomcnt,4.160569e-04,0.204116,1.348342e-05
censustractandblock,4.729801e-03,0.398118,5.539151e-03
airconditioningtypeid_5,-4.510281e-15,0.050538,-1.651457e-15
heatingorsystemtypeid_2,1.096900e-13,0.939773,-5.367928e-14
heatingorsystemtypeid_3,-2.014014e-15,0.026755,-6.869505e-16


A birdie from another script suggested these columns to use instead of the entire dataset. 

This might increase the algorithm performance. 

In [34]:
print(kmeans1.labels_[kmeans1.labels_ == 0].size)
print(kmeans1.labels_[kmeans1.labels_ == 1].size)
print(kmeans1.labels_[kmeans1.labels_ == 2].size)

11617
27247
28842


In [44]:
kmeans1.labels_.shape

(67706,)

In [48]:
print(y_train.loc[kmeans1.labels_ == 0].mean())
print(y_train.loc[kmeans1.labels_ == 1].mean())
print(y_train.loc[kmeans1.labels_ == 2].mean())

0.008011397090470851
0.01129679597753867
0.013686953054573157


In [52]:
print(y_train.loc[kmeans1.labels_ == 0].std())
print(y_train.loc[kmeans1.labels_ == 1].std())
print(y_train.loc[kmeans1.labels_ == 2].std())

0.18854920432348077
0.16354896784540066
0.15413461693601999


In [35]:
from sklearn import metrics 

In [42]:
homogenity = metrics.homogeneity_score(y_train, kmeans1.labels_)
v_meas = metrics.v_measure_score(y_train, kmeans1.labels_)
print("Homogenity measure: ", homogenity)
print("V-Score: ", v_meas)

Homogenity measure:  0.00584874271088
V-Score:  0.00991747663403


### K-Means with 2 Clusters 

We examine distance between the center of the cluster in each dimension.  
We also look at the variables where the center is farthest apart. 

In [43]:
kmeans2 = KMeans(n_clusters = 2).fit(train_df4)
cluster_centers2 = pd.DataFrame(kmeans2.cluster_centers_, columns = train_df4.columns, index = [1,2])
cluster_centers2.T[(cluster_centers2.T[1] - cluster_centers2.T[2]) > 0.02]

,1,2
latitude,0.520793,3.478554e-01
regionidcounty,1.000000,2.133130e-01
airconditioningtypeid_0,0.999981,9.419637e-01
heatingorsystemtypeid_1,0.710842,-1.951772e-13
heatingorsystemtypeid_4,0.287358,9.617763e-04
propertylandusetypeid_6,0.711176,6.132285e-01
propertylandusetypeid_12,0.040476,4.204336e-03


We will need to look at the data dictionary to determine which flag variables are creating the separation. 

In [39]:
print(kmeans2.labels_[kmeans2.labels_ == 0].size)
print(kmeans2.labels_[kmeans2.labels_ == 1].size)

36391
53884


In [50]:
print(Y.loc[kmeans2.labels_ == 0].mean())
print(Y.loc[kmeans2.labels_ == 1].mean())

0.011519657041050957
0.011364768761506888


In [51]:
print(Y.loc[kmeans2.labels_ == 0].std())
print(Y.loc[kmeans2.labels_ == 1].std())

0.16042427001369836
0.16204535596367725


In [40]:
print("Cluster 1 Percentage: " , kmeans2.labels_[kmeans2.labels_ == 0].size/ kmeans2.labels_.size)

Cluster 1 Percentage:  0.4031127111603434
